In [ ]:
! pip install -U langchain PyPDF2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00


In [ ]:
! pip install -U InstructorEmbedding faiss-cpu sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 147.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.5 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub

In [ ]:
def get_pdf_text(pdf_docs):
  text = ""
  for pdf in pdf_docs:
    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
      text += page.extract_text()
    return text

In [ ]:
def get_text_chunks(text):
  text_splitter = CharacterTextSplitter(
      #seperator='\n\n',
      chunk_size=1000,
      chunk_overlap = 200,
      length_function = len
  )
  chunks = text_splitter.split_text(text)
  return chunks

In [ ]:
def get_vectorstore(text_chunks):
  embeddings = HuggingFaceInstructEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  vectorstore = FAISS.from_texts(texts=text_chunks,
                                 embedding=embeddings)
  return vectorstore

In [ ]:
def get_conversation_chain(vectorstore):
  llm = HuggingFaceHub(huggingfacehub_api_token = '....',repo_id='google/flan-t5-base',
                       model_kwargs={"temperature":0.5,
                                     "max_length":512})
  memory = ConversationBufferMemory(memory_key='chat_history',
                                    return_messages=True)
  conversation_chain = ConversationalRetrievalChain.from_llm(
      llm = llm,
      retriever = vectorstore.as_retriever(),
      memory=memory
  )
  return conversation_chain

In [ ]:
def handle_userinput(user_question, conversation):
    response = conversation({'question':user_question})
    chat_history = response.get('chat_history', [])
    if not chat_history:
        chat_history = []
    for i, message in enumerate(chat_history):
        if i % 2 == 0:
            print(f"User:{message.content}")
        else:
            print(f"Bot:{message.content}")


In [ ]:
def main():
    # process uploaded pdfs
    pdf_docs = input("Enter path to pdfs separated by comma:").split(",")

    # create vectorstore
    text_chunks = get_text_chunks(get_pdf_text(pdf_docs))
    vectorstore = get_vectorstore(text_chunks)

    # create conversation chain
    conversation = get_conversation_chain(vectorstore)

    # ask a question
    user_question = input("Ask a question about your document:")
    if user_question:
        handle_userinput(user_question, conversation)


if __name__ == '__main__':
  main()

Enter path to pdfs separated by comma:/content/CG-Unit 8.pdf
load INSTRUCTOR_Transformer
max_seq_length  512
Ask a question about your document:What are some surface rendering techniques?
User:What are some surface rendering techniques?
Bot:Constant intensity shading method Gouraud shading method Phong shading mehod (Normal vector interpolation)
